In [71]:
import numpy as np
import pandas as pd
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def printVar (varname, var):

    print(varname)
    print(var)

def saveArray (name, array):

    pd.DataFrame(array).to_csv("savedVariables/" + name + ".csv", index=False, header=None)



In [72]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

Lx = .01 #width of domain
Ly = .01 #height of domain

fac =0

tr_left_x = 0     #u boundary condition at the left boundary
tr_left_y = 0    #v boundary condition at the left boundary

tr_right_x = 0    #u boundary condition at the right boundary
tr_right_y = - 100    #v boundary condition at the right boundary

tr_top_x = 0       #u boundary condition at the top boundary
tr_top_y = 0       #v boundary condition at the top boundary

tr_bottom_x = 0    #u boundary condition at the bottom boundary 
tr_bottom_y = 0    #v boundary condition at the bottom boundary

nx = 8            #number of control volumes in the x-direction
ny = 8            #number of control volumes in the y-direction

tr_left_x


0

In [73]:
# Define material properties

rho = 8050         #density of steel in kg/m^3
mu = 78*10e9       #shear modulus in Pa
K = 1.6*10e11      #bulk modulus in Pa

Lambda = K - (2/3)*mu

#print(Lambda)

In [74]:
## Define required variables

tf = 0.01           #total time in seconds
dx = Lx/nx          #length of each control volume in the x-direction
dy = Ly/ny          #length of each control volume in the y-direction
dt = tf/10          #size of time steps
Sfx = dy            #area vector x component (Area of East and West Faces)
Sfy = dx            #area vector y component (Area of North and South Faces)


In [75]:
## Define  position and time vectors

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
x = np.zeros((1,nx+2))   #position vector defining the center points of the control volumes in the x-direction.
x[0,nx+1] = Lx
x[0,1:nx+1] = np.arange(dx/2,Lx,dx)

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
y = np.zeros((1,ny+2))   #position vector defining the center points of the control volumes in the y-direction.
y[0,ny+1] = Ly
y[0,1:ny+1] = np.arange(dy/2,Ly,dy)

In [76]:
## Define required displacement matrices and b vector

u_new = np.zeros([(ny+2)*(nx+2),2])         #unknown displacements at t + 1    (Column 1 = x, Column2 = y)
u_old = np.zeros([(ny+2)*(nx+2),2])         #displacement at time t
u_old_old = np.zeros([(ny+2)*(nx+2),2])     #displacement at time t - 1

#rhs vector
b_x = np.zeros([(ny+2)*(nx+2),1])
b_y = np.zeros([(ny+2)*(nx+2),1])

## X equation:

In [77]:
class A:

    val = "value that stays in class without changning with instance"

    def __init__(self, xy):

        if xy == "x":

            self.a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((2*mu)*Sfy)/dy
            self.a_N = ((mu)*Sfy)/dy
            self.a_S = ((mu)*Sfy)/dy
            self.a_E = ((2*mu + Lambda)*Sfx)/dx
            self.a_W = ((2*mu + Lambda)*Sfx)/dx

        if xy == "y":

            self.a_P = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfy)/dy + ((2*mu)*Sfx)/dx 
            self.a_N = ((2*mu + Lambda)*Sfy)/dy
            self.a_S = ((2*mu + Lambda)*Sfy)/dy
            self.a_E = ((mu)*Sfx)/dx
            self.a_W = ((mu)*Sfx)/dx          

    def createMatrix(self):

        A_no_boundary = np.zeros([(ny+2)*(nx+2), (ny+2)*(nx+2)])
        # A_no_boundary = csr_matrix(A_no_boundary)  

        for j in np.arange(2, ny):     

            for k in np.arange((j*(nx+2)+2), (j*(nx+2)+nx)):  
                
                A_no_boundary[k,k] = self.a_P
                A_no_boundary[k,k+nx+2] = -self.a_N
                A_no_boundary[k,k-nx-2] = -self.a_S
                A_no_boundary[k,k+1] = -self.a_E
                A_no_boundary[k,k-1] = -self.a_W
            
        return A_no_boundary






In [78]:
# linear indexing

class index:

    def __init__(self, k):

        n = nx + 2
        s = -nx -2
        e = + 1
        w = - 1
        ne =  n + e
        se = s + e
        sw = s + w
        nw = n + w

        self.n = k + n
        self.s = k + s
        self.e = k + e
        self.w = k + w
        self.ne = k + ne
        self.se = k + se
        self.sw = k + sw
        self.nw = k + nw

    def dir(self, directionarray):

        n = nx + 2
        s = -nx -2
        e = + 1
        w = - 1
        ne =  n + e
        se = s + e
        sw = s + w
        nw = n + w
        
        arr1 = np.array([n, s, e, w, ne, se, sw, nw])
        arr2 = np.array(["n", "s", "e", "w", "ne", "se", "sw", "nw"])

        direction = k
        
        for i in np.arange(0, len(directionarray)):

            for j in np.arange(0, len(arr2)):    

                if arr2[j] == directionarray[i]:

                    direction = direction + arr1[j]

        return direction


k = 0

index(k).dir(["n", "n"])

20

In [79]:
class cell_index:

    bottom_left_corner_val = nx+3
    bottom_right_corner_val = 2*(nx+2) -2
    top_left_corner_val = (nx+2)*(ny+2) - 2*(nx+2) +1
    top_right_corner_val = (nx+2)*(ny+2) - (nx+2) -2

    def bottom_left_corner(self, k):
        if k == cell_index.bottom_left_corner_val:
            return True
        else:
            return False
    
    def bottom_right_corner(self, k):
        if k == cell_index.bottom_left_corner_val:
            return True
        else:
            return False
    
    def top_left_corner(self, k):
        if k == cell_index.bottom_left_corner_val:
            return True
        else:
            return False

    def top_right_corner(self, k):
        if k == cell_index.bottom_left_corner_val:
            return True
        else:
            return False

    def center_bottom(self, k):

        if (k > cell_index.bottom_left_corner_val)\
             & (k < cell_index.bottom_right_corner_val):
            return True
        else:
            return False

    def center_top(self, k):

        if (k > cell_index.top_left_corner_val)\
             & (k < cell_index.top_right_corner_val):
            return True
        else:
            return False

    def center_left(self, k):

        if (k < cell_index.top_left_corner_val)\
            & (k > cell_index.bottom_right_corner_val)\
                & (k%(nx+2) == 1):
            return True
        else:
            return False 

    def center_right(self, k):

        if (k < cell_index.top_left_corner_val)\
            & (k > cell_index.bottom_right_corner_val)\
                & (k%(nx+2) == nx):
            return True
        else:
            return False 

for k in np.arange(0,99):
    # print(k)
    if cell_index().center_top(k):
        print(k)

82
83
84
85
86
87


In [80]:
class boundary_point_index:

    bottom_left_corner_val = 0
    bottom_right_corner_val = nx + 1
    top_left_corner_val = (nx+2)*(ny +1) 
    top_right_corner_val = (nx+2)*(ny +2) - 1

    bottom_left_bottom_val = bottom_left_corner_val + 1
    bottom_left_left_val = bottom_left_corner_val + nx + 2
    bottom_right_bottom_val = bottom_right_corner_val - 1
    bottom_right_right_val = bottom_right_corner_val + nx + 2
    top_left_left_val = top_left_corner_val - (nx + 2)
    top_left_top_val = top_left_corner_val + 1
    top_right_right_val = top_right_corner_val - (nx + 2)
    top_right_top_val = top_right_corner_val - 1

    def bottom_left_bottom(self, k): 
        if k == boundary_point_index.bottom_left_bottom_val:
            return True
        else:
            return False

    def bottom_left_left(self, k): 
        if k == boundary_point_index.bottom_left_left_val:
            return True
        else:
            return False

    def bottom_right_bottom(self, k):
        if k == boundary_point_index.bottom_right_bottom_val:
            return True
        else:
            return False

    def bottom_right_right(self, k):
        if k == boundary_point_index.bottom_right_right_val:
            return True
        else:
            return False

    def top_left_left(self, k):
        if k == boundary_point_index.top_left_left_val:
            return True
        else:
            return False

    def top_left_top(self, k):
        if k == boundary_point_index.top_left_top_val:
            return True
        else:
            return False

    def top_right_right(self, k):
        if k == boundary_point_index.top_right_right_val:
            return True
        else:
            return False
    def top_right_top(self, k):
        if k == boundary_point_index.top_right_top_val:
            return True
        else:
            return False

    def bottom_left_corner(self, k):
        if k == boundary_point_index.bottom_left_corner_val:
            return True
        else:
            return False
    
    def bottom_right_corner(self, k):
        if k == boundary_point_index.bottom_right_corner_val:
            return True
        else:
            return False
    
    def top_left_corner(self, k):
        if k == boundary_point_index.top_left_corner_val:
            return True
        else:
            return False

    def top_right_corner(self, k):
        if k == boundary_point_index.top_right_corner_val:
            return True
        else:
            return False

    def center_bottom(self, k):

        if (k > boundary_point_index.bottom_left_bottom_val)\
             & (k < boundary_point_index.bottom_right_bottom_val):
            return True
        else:
            return False

    def center_top(self, k):

        if (k > boundary_point_index.top_left_top_val)\
             & (k < boundary_point_index.top_right_top_val):
            return True
        else:
            return False

    def center_left(self, k):

        if (k < boundary_point_index.top_left_left_val)\
            & (k > boundary_point_index.bottom_left_left_val)\
                & (k%(nx+2) == 0):
            return True
        else:
            return False 

    def center_right(self, k):

        if (k < boundary_point_index.top_right_right_val)\
            & (k > boundary_point_index.bottom_right_right_val)\
                & (k%(nx+2) == nx+1):
            return True
        else:
            return False 

for k in np.arange(0,99):
    # print(k)
    if boundary_point_index().center_bottom(k):
        print(k)

2
3
4
5
6
7


In [81]:
# sys.path.append("./Classes/")

# from boundary_point_index import boundary_point_index

# boundary_point_index().nx = nx
# boundary_point_index().ny = nx

In [82]:
class cell_index:

    bottom_left_corner_val = nx+3
    bottom_right_corner_val = 2*(nx+2) -2
    top_left_corner_val = (nx+2)*(ny+2) - 2*(nx+2) +1
    top_right_corner_val = (nx+2)*(ny+2) - (nx+2) -2

    def bottom_left_corner(self, k):
        if k == cell_index.bottom_left_corner_val:
            return True
        else:
            return False
    
    def bottom_right_corner(self, k):
        if k == cell_index.bottom_right_corner_val:
            return True
        else:
            return False
    
    def top_left_corner(self, k):
        if k == cell_index.top_left_corner_val:
            return True
        else:
            return False

    def top_right_corner(self, k):
        if k == cell_index.top_right_corner_val:
            return True
        else:
            return False

    def center_bottom(self, k):

        if (k > cell_index.bottom_left_corner_val)\
             & (k < cell_index.bottom_right_corner_val):
            return True
        else:
            return False

    def center_top(self, k):

        if (k > cell_index.top_left_corner_val)\
             & (k < cell_index.top_right_corner_val):
            return True
        else:
            return False

    def center_left(self, k):

        if (k < cell_index.top_left_corner_val)\
            & (k > cell_index.bottom_right_corner_val)\
                & (k%(nx+2) == 1):
            return True
        else:
            return False 

    def center_right(self, k):

        if (k < cell_index.top_left_corner_val)\
            & (k > cell_index.bottom_right_corner_val)\
                & (k%(nx+2) == nx):
            return True
        else:
            return False 

# for k in np.arange(0,99):
#     # print(k)
#     if cell_index().center_top(k):
#         print(k)

In [83]:
class displacement(boundary_point_index, cell_index):

    # add a statement using indexes

    def __init__(self, k, u_previous, direction):   

        if k > boundary_point_index().bottom_right_corner_val:
            self.S = u_previous[k-(nx+2), direction]

        if k < boundary_point_index().top_left_corner_val:
            self.N = u_previous[k+(nx+2), direction]
        
        if k > boundary_point_index().bottom_left_corner_val:
            self.W = u_previous[k-1, direction]
        
        if k < boundary_point_index().top_right_corner_val:
            self.E = u_previous[k+1, direction]

        if k >= cell_index().bottom_left_corner_val:
            self.SE = u_previous[k-(nx+2)+1, direction]
        
        if k >= boundary_point_index().bottom_left_left_val:
            self.SW = u_previous[k-(nx+2)-1, direction]

        if k <= cell_index().top_right_corner_val:
            self.NE = u_previous[k+(nx+2)+1, direction]
        
        if k <= boundary_point_index().top_right_right_val:
            self.NW = u_previous[k+(nx+2)-1, direction]    


V = displacement(k, u_new, 1)

for k in np.arange(0,(nx+2)*(ny+2)):  
    if k == 22:
        V = displacement(k, u_new, 1)
        print(k)
        print(V.SE)


boundary_point_index().top_right_corner_val

22
0.0


99

In [84]:
## Time loop

!rm -r savedVariables
!mkdir savedVariables

for time in t:

    print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 10000

    # Make directory

    !mkdir savedVariables/{time}
    
    while True:
    # for iteration in [1]:
            
        u_previous = u_new #store latest guess values

        A_x = A("x").createMatrix()
        
        for k in np.arange(0,(nx+2)*(ny+2)):   # k is the cell number

            U = displacement(k, u_previous, 0)
            V = displacement(k, u_previous, 1)
               
            if cell_index().bottom_left_corner(k):

                #print(str(k) + ": Bottom left corner coefficients")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu

                #ae
                A_x[k,k+1] = - (Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V.NE
                                                + V.E
                                                - V.NW #l
                                                - V.W #l
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V.NE
                                                    + V.N
                                                    - V.SE #b
                                                    - V.S #b
                                                )
                            +
                            Sfy*tr_bottom_x
                            +
                            Sfx*tr_left_x
                        )   

            #Bottom right corner coefficients
                
            elif  cell_index().bottom_right_corner(k):

                #print(str(k) + ": Bottom right corner coefficients")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_b) + C(u_r) + D(-V.W -V.NW + 2v_r) + E(-V.N -V.NW + 2v_b)


                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V.NE #r
                                                + V.E #r
                                                - V.NW
                                                - V.W
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V.N
                                                    - V.NW
                                                    + V.S #b
                                                    + V.SW #b
                                                )
                            +
                            Sfx*tr_right_x
                            +
                            Sfy*tr_bottom_x
                        ) 

            # Top Left Corner Coeficients
                
            elif  cell_index().top_left_corner(k):

                #print(str(k) + ":Top Left corner coefficients")

                #ap                
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)
                
                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #ae                
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)
                
                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V.E
                                                - V.SE
                                                + V.W #l
                                                + V.SW #l
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V.NE #t
                                                    + V.N #t
                                                    - V.SE
                                                    - V.S
                                                )
                            +
                            Sfy*tr_top_x
                            +
                            Sfx*tr_left_x
                        )
                
            # Top right corner coefficients
                
            elif  cell_index().top_right_corner(k):   

                #print(str(k) + ":Top right corner coefficients")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V.E #b
                                                - V.SE #b
                                                + V.W
                                                + V.SW
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V.N #t
                                                    - V.NW #t
                                                    + V.S
                                                    + V.SW
                                                )
                            +
                            Sfy*tr_top_x
                            +
                            Sfx*tr_right_x
                        )
                
            # Center Bottom Boundaries

            elif  cell_index().center_bottom(k):

                #print(str(k) + ": Bottom Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda) + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu
                
                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(4*dx))*mu*(
                                                + V.NE
                                                + V.E
                                                - V.NW
                                                - V.W
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V.NE
                                                    + V.N
                                                    - V.SE #b
                                                    - V.S #b
                                                )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V.N
                                                    - V.NW
                                                    + V.S #b
                                                    + V.SW #b
                                                )
                            +
                            Sfy*tr_bottom_x
                        )                                 

            # Center Top Boundaries

            elif cell_index().center_top(k):

                # #print(k)

                #print(str(k) + ": Top Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda) + (Sfx/dx)*(2*mu + Lambda)

                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu
                
                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(4*dx))*mu*(
                                                - V.E
                                                - V.SE
                                                + V.W
                                                + V.SW
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V.NE #t
                                                    + V.N #t
                                                    - V.SE
                                                    - V.S
                                                )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V.N #t
                                                    - V.NW #t
                                                    + V.S
                                                    + V.SW
                                                )
                            +
                            Sfy*tr_top_x
                        )
            
            # Center Left Boundaries

            elif  cell_index().center_left(k):

                #print(str(k) + ": Left Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu
                
                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V.NE
                                                + V.E
                                                - V.NW #l
                                                - V.W #l
                                            )
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V.E
                                                - V.SE
                                                + V.W #l
                                                + V.SW #l 
                                            )
                            +
                            (Sfx/(4*dy))*Lambda*(
                                                   + V.NE
                                                   + V.N
                                                   - V.SE
                                                   - V.S
                                                )
                            +
                            Sfx*tr_left_x
                        )

            # Center Right Boundaries

            elif  cell_index().center_right(k):

                #print(str(k) + ": Right Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu
                
                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V.NE #r
                                                + V.E #r
                                                - V.NW 
                                                - V.W 
                                            )
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V.E #r
                                                - V.SE #r
                                                + V.W 
                                                + V.SW 
                                            )
                            +
                            (Sfx/(4*dy))*Lambda*(
                                                   - V.N 
                                                   - V.NW
                                                   + V.S
                                                   + V.SW
                                                )
                            +
                            Sfx*tr_right_x
                        )

            #### Points on Boundary ####

            # Center Bottom

            elif boundary_point_index().center_bottom(k):

                #print(str(k) +  ": Bottom points")

                #ap
                A_x[k,k] = 2*mu/dy

                #an
                A_x[k,k+nx+2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_bottom_x
                        +
                        (mu/(2*dx))*(
                                        + V.E
                                        - V.W
                                    )
                        ) 

            # Center Top

            elif boundary_point_index().center_top(k):
                
                #print( str(k) + ": Center points" )
                
                #ap
                A_x[k,k] = 2*mu/dy

                #as
                A_x[k,k-nx-2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_top_x
                        +
                        (mu/(2*dx))*(
                                        - V.E
                                        + V.W
                                    )
                        ) 

                
            # Center Left

            elif boundary_point_index().center_left(k):

                #print( str(k) + ": Left points" )

                # #ap
                # A_x[k,k] = (4*mu + 2*Lambda)/dx

                # #ae
                # A_x[k,k+1] = -(4*mu + 2*Lambda)/dx

                # #b
                # b_x[k] =(
                #         tr_left_x
                #         +
                #         (Lambda/(2*dy))*(
                #                         + V.N
                #                         - V.S
                #                     )
                #         )
                A_x[k,k] = 1
                b_x[k] = 1e-100

            # Center Right

            elif boundary_point_index().center_right(k):

                #print( str(k) + ": Right points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #aw
                A_x[k,k-1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_right_x
                        +
                        (Lambda/(2*dy))*(
                                        - V.N
                                        + V.S
                                    )
                        )

            # Bottom Left: Bottom

            elif boundary_point_index().bottom_left_bottom(k):

                # print( str(k) + ": Bottom Left: Bottom points" )
                
                # ap
                A_x[k,k] = 2*mu/dy

                #an
                A_x[k,k+nx+2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_bottom_x
                        +
                        ((2*mu)/(3*dx))*(
                                        + V.E
                                        - V.W
                                    )
                        ) 

            # Bottom Left: Left

            elif boundary_point_index().bottom_left_left(k):

                #print( str(k) +  ": Bottom Left: Left points" )
                
                # #ap
                # A_x[k,k] = (4*mu + 2*Lambda)/dx

                # #ae
                # A_x[k,k+1] = -(4*mu + 2*Lambda)/dx

                # #b
                # b_x[k] =(
                #         tr_left_x
                #         +
                #         ((2*Lambda)/(3*dy))*(
                #                         + V.N
                #                         - V.S
                #                     )
                #         )

                A_x[k,k] = 1
                b_x[k] = 1e-100
            
            # Bottom Right: Right

            elif boundary_point_index().bottom_right_right(k):

                #print( str(k) + ": Bottom Right: Right points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #aw
                A_x[k,k-1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_right_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                        - V.N
                                        + V.S
                                    )
                        )

            # Bottom Right: Bottom

            elif boundary_point_index().bottom_right_bottom(k):

                #print( str(k) + ": Bottom Right: Bottom points" )


                #ap
                A_x[k,k] = 2*mu/dy

                #an
                A_x[k,k+nx+2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_bottom_x
                        +
                        ((2*mu)/(3*dx))*(
                                        + V.E
                                        - V.W
                                    )
                        )

            # Top Left: Left

            elif boundary_point_index().top_left_left(k):

                #print( str(k) + " Top Left: Left points" )

                
                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #ae
                A_x[k,k+1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_left_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                        + V.N
                                        - V.S
                                    )
                        )

            # Top Left: Top

            elif boundary_point_index().top_left_top(k):

                #print( str(k) + " Top Left: Top points" )

                
                #ap
                A_x[k,k] = 2*mu/dy

                #as
                A_x[k,k-nx-2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_top_x
                        +
                        ((2*mu)/(3*dx))*(
                                        - V.E
                                        + V.W
                                    )
                        )

            # Top Right: Right

            elif boundary_point_index().top_right_right(k):

                #print( str(k) + " Top Right: Right points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #aw
                A_x[k,k-1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_right_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                                - V.N
                                                + V.S
                                            )
                        )

            # Top Right: Top
            
            elif boundary_point_index().top_right_top(k):

                #print( str(k) + " Top Right: Top points" )

                #ap
                A_x[k,k] = 2*mu/dy

                #as
                A_x[k,k-nx-2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_top_x
                        +
                        ((2*mu)/(3*dx))*(
                                            - V.E
                                            + V.W
                                    )
                        ) 
            # Corner Points

            elif boundary_point_index().bottom_left_corner(k):

                #print(str(k) + ": Bottom Left Corner Point on Boundary")

                A_x[k,k] = 4

                A_x[k,index(k).n] = 2 
                A_x[k,index(k).e] = 2

                A_x[k,index(k).ne] = -15

                A_x[k,index(k).dir(["n", "n"])] = 2 
                A_x[k,index(k).dir(["e", "e"])] = 2

                A_x[k,index(k).dir(["n", "ne"])] = 1
                A_x[k,index(k).dir(["ne", "ne"])] = 1
                A_x[k,index(k).dir(["e", "ne"])] = 1

                b_x[k] = 0

                # A_x[k,k] = 1
                # b_x[k] = 1e-100

            elif boundary_point_index().bottom_right_corner(k):
                #print(str(k) + ": Bottom Right Corner Point on Boundary")
                A_x[k,k] = 4

                A_x[k,index(k).n] = 2 
                A_x[k,index(k).w] = 2

                A_x[k,index(k).nw] = -15

                A_x[k,index(k).dir(["n", "n"])] = 2 
                A_x[k,index(k).dir(["w", "w"])] = 2

                A_x[k,index(k).dir(["n", "nw"])] = 1
                A_x[k,index(k).dir(["nw", "nw"])] = 1
                A_x[k,index(k).dir(["w", "nw"])] = 1

                b_x[k] = 0

            elif boundary_point_index().top_left_corner(k):
                #print(str(k) + ": Top Left Corner Point on Boundary")
                A_x[k,k] = 4

                A_x[k,index(k).s] = 2 
                A_x[k,index(k).e] = 2

                A_x[k,index(k).se] = -15

                A_x[k,index(k).dir(["s", "s"])] = 2 
                A_x[k,index(k).dir(["e", "e"])] = 2

                A_x[k,index(k).dir(["s", "se"])] = 1
                A_x[k,index(k).dir(["se", "se"])] = 1
                A_x[k,index(k).dir(["e", "se"])] = 1

                b_x[k] = 0

                # A_x[k,k] = 1
                # b_x[k] = 1e-100

            elif boundary_point_index().top_right_corner(k):
                #print(str(k) + ": Top Right Corner Point on Boundary")
                A_x[k,k] = 4

                A_x[k,index(k).s] = 2 
                A_x[k,index(k).w] = 2

                A_x[k,index(k).sw] = -15

                A_x[k,index(k).dir(["s", "s"])] = 2 
                A_x[k,index(k).dir(["w", "w"])] = 2

                A_x[k,index(k).dir(["s", "sw"])] = 1
                A_x[k,index(k).dir(["se", "sw"])] = 1
                A_x[k,index(k).dir(["w", "sw"])] = 1

                b_x[k] = 0
                
            else:

                # print(str(k) + ": NO BC Cell")

                b_x[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy)
                            + (Sfy/(4*dx))*mu*(
                                                + V.NE    # + V.Ne
                                                - V.NW    # - V.NW
                                                - V.SE    # - V.SE
                                                + V.SW    # + V.SW
                                            )
                            +(Sfx/(4*dy))*Lambda*(
                                                    + V.NE    # + V.Ne
                                                    - V.NW    # - V.NW
                                                    - V.SE    # - V.SE
                                                    + V.SW    # + V.SW 
                                                )
                            )

        # saveArray(str(time) + "/A_x_" + str(icorr) , A_x.todense())   
        # saveArray(str(time) + "/b_x_" + str(icorr), b_x)  

        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        # #print(Ux)

        saveArray(str(time) + "/Ux_" + str(icorr), Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A("y").createMatrix() #make copy of A with no boundary conditions
        
        for k in np.arange(0,(nx+2)*(ny+2)):   # j is the cell number


            U = displacement(k, u_previous, 0)
            V = displacement(k, u_previous, 1)
 
            #Bottom left corner coefficients
            
            if k == nx+3:

                #print(str(k) + ": Bottom left corner coefficients")
                
                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)

                #ae                
                A_y[k,k+1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U.NE
                                                + U.E
                                                - U.NW #l
                                                - U.W #l
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U.NE
                                                    + U.N
                                                    - U.SE #b
                                                    - U.S #b
                                                )
                            +
                            Sfy*tr_bottom_y
                            +
                            Sfx*tr_left_y
                        )
                
            #Bottom right corner coefficients

            elif  k == 2*(nx+2) -2:
                
                #print(str(k) + ": Bottom right corner coefficients")
               
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)

                #aw                
                A_y[k,k-1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U.NE #r
                                                + U.E #r
                                                - U.NW
                                                - U.W
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U.N
                                                    - U.NW
                                                    + U.S #b
                                                    + U.SW #b
                                                )
                            +
                            Sfy*tr_bottom_y
                            +
                            Sfx*tr_right_y
                        )
                     
            #Top left corner coefficients
                
            elif  k == (nx+2)*(ny+2) - 2*(nx+2) +1:

                #print(str(k) + ":Top Left corner coefficients")

                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #ae                
                A_y[k,k+1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U.E
                                                - U.SE
                                                + U.W #l
                                                + U.SW #l
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U.NE #t
                                                    + U.N #t
                                                    - U.SE
                                                    - U.S
                                                )
                            +
                            Sfy*tr_top_y
                            +
                            Sfx*tr_left_y
                        )    
                
            #Top right corner coefficients
                
            elif  k == (nx+2)*(ny+2) - (nx+2) -2:   

                #print(str(k) + ":Top right corner coefficients")

                #ap
                
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #aw                
                A_y[k,k-1] = -(Sfx/dx)*(mu)
                
                b_y[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U.E #r
                                                - U.SE #r
                                                + U.W
                                                + U.SW
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U.N #t
                                                    - U.NW #t
                                                    + U.S
                                                    + U.SW
                                                )
                            +
                            Sfy*tr_top_y
                            +
                            Sfx*tr_right_y
                        )
            
            # Center Bottom Boundaries

            elif  (k > nx+3) & (k < 2*(nx+2) -1):

                #print(str(k) + ": Bottom Center Coefficents for Cell")


                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu) + (Sfx/dx)*(mu)

                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(4*dx))*Lambda*(
                                                + U.NE
                                                + U.E
                                                - U.NW
                                                - U.W
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U.NE
                                                    + U.N
                                                    - U.SE #b
                                                    - U.S #b
                                                )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U.N
                                                    - U.NW
                                                    + U.S #b
                                                    + U.SW #b
                                                )
                            +
                            Sfy*tr_bottom_y
                        )

            # Center Top Boundaries

            elif  ( k > (nx+2)*(ny+2) - 2*(nx+2) +1 ) & ( k < (nx+2)*(ny+2) - (nx+2) -2 ):

                #print(str(k) + ": Top Center Coefficents for Cell")

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu) + (Sfx/dx)*(mu)

                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(4*dx))*Lambda*(
                                                - U.E
                                                - U.SE
                                                + U.W
                                                + U.SW
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U.NE #t
                                                    + U.N #t
                                                    - U.SE
                                                    - U.S
                                                )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U.N #t
                                                    - U.NW #t
                                                    + U.S
                                                    + U.SW
                                                )
                            +
                            Sfy*tr_top_y
                        )

            # Center Left Boundaries

            elif  ( k > nx+3) & ( k%(nx+2) == 1 ) & ( k < (nx+2)*(ny+2) - 2*(nx+2) +1):

                #print(str(k) + ": Left Center Coefficents for Cell")

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*mu

                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U.NE
                                                + U.E
                                                - U.NW #l
                                                - U.W #l
                                            )
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U.E
                                                - U.SE
                                                + U.W #l
                                                + U.SW #l 
                                            )
                            +
                            (Sfx/(4*dy))*mu*(
                                                   + U.NE
                                                   + U.N
                                                   - U.SE
                                                   - U.S
                                                )
                            +
                            Sfx*tr_left_y
                        )

            # Center Right Boundaries

            elif  ( k > 2*(nx+2) -2) & ( k%(nx+2) == nx ) & ( k < (nx+2)*(ny+2) - (nx+2) -2 ):

                #print(str(k) + ": Right Center Coefficents for Cell")

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*mu

                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U.NE #r
                                                + U.E #r
                                                - U.NW 
                                                - U.W 
                                            )
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U.E #r
                                                - U.SE #r
                                                + U.W 
                                                + U.SW 
                                            )
                            +
                            (Sfx/(4*dy))*mu*(
                                                   - U.N 
                                                   - U.NW
                                                   + U.S
                                                   + U.SW
                                                )
                            +
                            Sfx*tr_right_y
                        )


            #### Points on Boundary ####

            # Center Bottom

            elif (k > 1) & (k < nx):

                #print(str(k) +  ": Bottom points")

                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #an
                A_y[k,k+nx+2] = -(4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_bottom_y
                        +
                        (Lambda/(2*dx))*(
                                        + U.E
                                        - U.W
                                    )
                        ) 

            # Center Top

            elif ( k >= ((nx+2)*(ny+2) - nx) ) & ( k < (nx+2)*(ny+2) - 2) :
                
                #print( str(k) + ": Center points" )
                
                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #as
                A_y[k,k-nx-2] = - (4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_top_y
                        +
                        (Lambda/(2*dx))*(
                                        - U.E
                                        + U.W
                                    )
                        ) 
                
            # Center Left

            elif ( k > 2*(nx+2) -1) & (k < (ny)*(nx+2) ) & (k%(nx+2) == 0):

                #print( str(k) + ": Left points" )

                # #ap
                # A_y[k,k] = (2*mu)/dx

                # #ae
                # A_y[k,k+1] = -(2*mu)/dx

                # #b
                # b_y[k] =(
                #         tr_left_y
                #         +
                #         (mu/(2*dy))*(
                #                         + U.N
                #                         - U.S
                #                     )
                #         )

                A_y[k,k] = 1
                b_y[k] = 1e-100 

            # Center Right

            elif ( k > 2*(nx+2) -1) & (k < (ny)*(nx+2)) & (k%(nx+2) == nx +1):

                #print( str(k) + ": Right points" )

                #ap
                A_y[k,k] = (2*mu)/dx

                #aw
                A_y[k,k-1] = -(2*mu)/dx

                #b
                b_y[k] =(
                        tr_right_x
                        +
                        (mu/(2*dy))*(
                                        - U.N
                                        + U.S
                                    )
                        )

            # Bottom Left: Bottom

            elif (k == 1):

                #print( str(k) + ": Bottom Left: Bottom points" )
                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dx      

                #an
                A_y[k,k+nx+2] = -(4*mu + 2*Lambda)/dx

                #b
                b_y[k] =(
                        tr_bottom_y
                        +
                        ((2*mu)/(3*dx))*(
                                        + U.E
                                        - U.W
                                    )
                        ) 
            # Bottom Left: Left

            elif (k == nx + 2):

                #print( str(k) +  ": Bottom Left: Left points" )
                
                # #ap
                # A_y[k,k] = (2*mu)/dx

                # #ae
                # A_y[k,k+1] = -(2*mu)/dx

                # #b
                # b_y[k] =(
                #         tr_left_y
                #         +
                #         ((2*mu)/(3*dy))*(
                #                         + U.N
                #                         - U.S
                #                     )
                #         )

                A_y[k,k] = 1
                b_y[k] = 1e-100
            
            # Bottom Right: Right

            elif ( k == 2*(nx+2) -1) :

                #print( str(k) + ": Bottom Right: Right points" )

                #ap
                A_y[k,k] = (2*mu)/dx

                #aw
                A_y[k,k-1] = -(2*mu)/dx

                #b
                b_y[k] =(
                        tr_right_y
                        +
                        ((2*mu)/(3*dy))*(
                                        - U.N
                                        + U.S
                                    )
                        )

            # Bottom Right: Bottom

            elif (k == nx) : 

                #print( str(k) + ": Bottom Right: Bottom points" )


                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #an
                A_y[k,k+nx+2] = -(4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_bottom_y
                        +
                        ((2*Lambda)/(3*dx))*(
                                        + U.E
                                        - U.W
                                    )
                        )

            # Top Left: Left

            elif ( k == (nx+2)*(ny+2) - 2*(nx+2) ) :

                #print( str(k) + " Top Left: Left points" )

                
                # #ap
                # A_y[k,k] = 2*mu/dx

                # #ae
                # A_y[k,k+1] = - 2*mu/dx

                # #b
                # b_y[k] =(
                #         tr_left_y
                #         +
                #         ((2*mu)/(3*dy))*(
                #                         + U.N
                #                         - U.S
                #                     )
                #         )

                A_y[k,k] = 1
                b_y[k] = 1e-100

            # Top Left: Top

            elif (k == (nx+2)*(ny+2) - (nx+2) +1):

                #print( str(k) + " Top Left: Top points" )

                
                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #as
                A_y[k,k-nx-2] = -(4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_top_y
                        +
                        ((2*Lambda)/(3*dx))*(
                                        - U.E
                                        + U.W
                                    )
                        )

            # Top Right: Right

            elif ( k == (nx+2)*(ny+2) - (nx+2) - 1) :

                #print( str(k) + " Top Right: Right points" )

                #ap
                A_y[k,k] =  2*mu/dx

                #aw
                A_y[k,k-1] = - 2*mu/dx

                #b
                b_y[k] =(
                        tr_right_y
                        +
                        ((2*mu)/(3*dy))*(
                                        - U.N
                                        + U.S
                                    )
                        )

            # Top Right: Top
            
            elif ( k == (nx+2)*(ny+2) - 2 ) : 

                #print( str(k) + " Top Right: Top points" )

                #ap
                A_y[k,k] = (4*Lambda + 2*mu)/dy

                #as
                A_y[k,k-nx-2] = -(4*Lambda + 2*mu)/dy

                #b
                b_y[k] =(
                        tr_top_y
                        +
                        ((2*Lambda)/(3*dx))*(
                                        - U.E
                                        + U.W
                                    )
                        ) 
            # Corner Points

            elif (k == 0):
                #print(str(k) + ": Bottom Left Corner Point on Boundary")

                A_y[k,k] = 1
                b_y[k] = 1e-100

            elif (k == nx + 1):
                #print(str(k) + ": Bottom Right Corner Point on Boundary")
                A_y[k,k] = 1
                b_y[k] = 1e-100

            elif (k == (nx+2)*(ny +1) ): 
                #print(str(k) + ": Top Left Corner Point on Boundary")
                A_y[k,k] = 1
                b_y[k] = 1e-100

            elif (k == (nx+2)*(ny +2) - 1) : 
                #print(str(k) + ": Top Right Corner Point on Boundary")
                A_y[k,k] = 1
                b_y[k] = 1e-100


            else:

                #print(str(k) + ": NO BC Cell")

                b_y[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                            + (Sfy/(4*dx))*Lambda*(
                                                + U.NE    # + U.Ne
                                                - U.NW    # - U.Nw
                                                - U.SE    # - U.Se
                                                + U.SW    # + U.Sw
                                            )
                            +(Sfx/(4*dy))*mu*(
                                                    + U.NE    # + U.Ne
                                                    - U.NW    # - U.Nw
                                                    - U.SE    # - U.Se
                                                    + U.SW    # + U.Sw 
                                                )
                            )
        # saveArray(str(time) + "/A_y_" + str(icorr) , A_y.todense())   
        # saveArray(str(time) + "/b_y_" + str(icorr), b_y)     

       
                
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        saveArray(str(time) + "/Uy_" + str(icorr), Uy)
        
        # ## Update U.New with new x and y displacements

        u_new1 = np.vstack((Ux, Uy)).T
        # saveArray (str(time) + "/U.New1_" + str(icorr), u_new1)    
        # saveArray (str(time) + "/u_previous_" + str(icorr), u_previous)

        # # U.New[:,0] = Ux
        # # U.New[:,1] = Uy
        
        ## Calculate the residual of each iteration
        
        normFactor = np.max(u_new1)
        residual = math.sqrt(np.mean((u_new1 - u_previous)**2))/normFactor
        tolerance = 1e-6

        u_new = u_new1
    
        ##print out residual of each iteration
        print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}".format(icorr, residual, normFactor))
             
        ## Convergence check
        
        if residual < tolerance:
            
            break
            
        elif icorr > maxcorr:
            
            break
            
        
        icorr = icorr + 1
    
    ## Update variables
    
    u_old_old = u_old
    u_old = u_new1
    
    # ## Add the boundary conditions to the solution matrices
    u_reshape_x = u_new[:,0].reshape(nx+2, ny+2)
    u_reshape_y = u_new[:,1].reshape(nx+2, ny+2)
    
    # #Boundary conditions for u matrix
    U = np.zeros((nx+2,ny+2))
    V = np.zeros((nx+2,ny+2))
    # U[0,1:nx+1] = u_bottom
    # U[1:ny+1,0] = u_left
    # U[1:ny+1,nx+1] = u_right
    # U[ny+1,1:nx+1] = u_top    
    U[:, :] = u_reshape_x
    V[:, :] = u_reshape_y
    
    # # #Boundary conditions for v matrix
    # V = np.zeros((nx+2,ny+2))
    # V[0,1:nx+1] = v_bottom
    # V[1:ny+1,0] = v_left
    # V[1:ny+1,nx+1] = v_right
    # V[ny+1,1:nx+1] = v_top    
    # V[1:nx+1,1:nx+1] = u_reshape_y  
    
    #Modulus of displacement


    disp_mod = np.sqrt((U**2) + (V**2))
    disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
#     # ## Plot the results

    import plotly.graph_objects as go

    disp_mod = np.sqrt((U**2) + (V**2))*1e-10

    dispx, dispy = np.meshgrid(x, y)
    fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])
    
fig.show()   




Time = 0.0
icorr = 1, Residual = 8240061730200895373414902282756591606158758254992071183982166998551923989026676239128755961856.00000000000000000000, normFactor = 0.00000000000000000000
icorr = 2, Residual = 0.22868955596126774865, normFactor = 0.00000108589518804487
icorr = 3, Residual = 0.13446434491015843427, normFactor = 0.00000108589518804487
icorr = 4, Residual = 0.08077144269043755320, normFactor = 0.00000132753992975878
icorr = 5, Residual = 0.07088694750902635455, normFactor = 0.00000132753992975878
icorr = 6, Residual = 0.05136861364748213782, normFactor = 0.00000140965429529428
icorr = 7, Residual = 0.04628135323969340131, normFactor = 0.00000140965429529428
icorr = 8, Residual = 0.03566254476346201324, normFactor = 0.00000145582486067924
icorr = 9, Residual = 0.03186371463108373520, normFactor = 0.00000145582486067924
icorr = 10, Residual = 0.02360166408289511922, normFactor = 0.00000158925710405655
icorr = 11, Residual = 0.02097360902414597064, normFactor = 0.000001589257